In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.nn import init
import copy
import numpy as np

import queue

In [2]:
gpu = False

In [3]:
class RND(nn.Module):
    def __init__(self,width = 240, height =256, channel = 3):
        super(RND,self).__init__()
        
        self.width = width
        self.height = height
        self.channel = channel
        
        self.predictor = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=32,
                kernel_size=8,
                stride=4),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=4,
                stride=2),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=64,
                out_channels=64,
                kernel_size=3,
                stride=1),
            nn.LeakyReLU(),
            Flatten(),
            nn.Linear(46592, 512), # change
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512)
            )
        self.target = copy.deepcopy(self.predictor)
        for p in self.modules():
            if isinstance(p, nn.Conv2d):
                init.orthogonal_(p.weight, np.sqrt(2))
                p.bias.data.zero_()

            if isinstance(p, nn.Linear):
                init.orthogonal_(p.weight, np.sqrt(2))
                p.bias.data.zero_()

        for param in self.target.parameters():
            param.requires_grad = False
            
    def forward(self, state):
        target_feature = self.target(state)
        predict_feature = self.predictor(state)
        return predict_feature, target_feature

In [4]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
class PPO(nn.Module):
    def __init__(self,width = 240, height =256, channel = 3, action_dim = 7):
        self.width = width
        self.height = height
        self.channel = channel
        self.action_dim = action_dim
        super(PPO,self).__init__()
        self.memory = []
        
        self.basic = nn.Sequential(\
                    nn.Conv2d(in_channels = 3,\
                             out_channels = 32,\
                             kernel_size = 8,\
                              stride = 4),
                    nn.ReLU(),
                                   nn.Conv2d(in_channels = 32,\
                                           out_channels = 64,\
                                           kernel_size = 4,\
                                           stride = 2),\
                    nn.ReLU(),\
                    nn.Conv2d(in_channels = 64,\
                             out_channels = 64,\
                             kernel_size = 3,\
                             stride = 1),
                                   nn.ReLU(),\
                                   Flatten(),
                    nn.Linear(46592,256), #have to change       
                    nn.ReLU(),\
                    nn.Linear(256,448),
                    nn.ReLU()
                    )
        self.actor = nn.Sequential(\
                                  nn.Linear(448,448),\
                                  nn.ReLU(),\
                                  nn.Linear(448,self.action_dim)\
                                  )
        
        self.extrinsic_critic = nn.Linear(448,1)
        self.intrinsic_critic = nn.Linear(448,1)
        
        init.orthogonal_(self.extrinsic_critic.weight, 0.01)
        self.extrinsic_critic.bias.data.zero_()

        init.orthogonal_(self.intrinsic_critic.weight, 0.01)
        self.intrinsic_critic.bias.data.zero_()

        for i in range(len(self.actor)):
            if type(self.actor[i]) == nn.Linear:
                init.orthogonal_(self.actor[i].weight, 0.01)
                self.actor[i].bias.data.zero_()
    def forward(self, x,dim = -1):
        x = self.basic(x)
        action = self.actor(x)
        action_prob = F.softmax(action,dim = dim)
        
        intrinsic = self.intrinsic_critic(x)
        extrinsic = self.extrinsic_critic(x)
        return action_prob,extrinsic,intrinsic    

    


In [41]:
learning_rate = 0.0001
gamma         = 0.98
lmbda         = 0.95
eps_clip      = 0.1
K_epoch       = 4
T_horizon     = 128
critic_coef = 0.5
ent_coef = 0.001
intrinsic_gamma = 0.99
extrinsic_gamma = 0.999
update_proportion = 0.25

extrinsic_advantage_coef = 2
intrinsic_advantage_coef = 1
class Agent(nn.Module):
    def __init__(self,width=240,height=256,channel = 3,action_dim=7,learning_rate=0.0005):
        self.width = width
        self.height = height
        self.channel = channel
        self.action_dim = action_dim
        
        super(Agent,self).__init__()
        
        self.memory = []
        self.intrinsic_queue = queue.Queue()
        self.intrinsic_input_queue = queue.Queue()
        self.ppo = PPO(self.width, self.height, self.channel, self.action_dim)
        self.rnd = RND(self.width, self.height , self.channel)
        self.optimizer = optim.Adam(self.parameters(),lr = learning_rate)
    def put_data(self,data):
        self.memory.append(data)
    def make_batch(self):
        state_list, action_list, extrinsic_reward_list, intrinsic_reward_list, next_state_list, \
        prob_list, extrinsic_done_list,intrinsic_done_list = [],[],[],[],[],[], [],[]
        for data in self.memory:
            state,action,extrinsic_reward, intrinsic_reward,next_state,prob,done = data
            state_list.append(state)
            action_list.append([action])
            extrinsic_reward_list.append([extrinsic_reward])
            intrinsic_reward_list.append([intrinsic_reward])

            next_state_list.append(next_state)
            extrinsic_done_mask = 0 if done else 1
            extrinsic_done_list.append([extrinsic_done_mask])
            intrinsic_done_list.append([1])
            prob_list.append([prob])
        self.memory = []

        s,a,er,ir,next_s,extrinsic_done_list,intrinsic_done_list,prob \
                                        = torch.tensor(state_list,dtype=torch.float),\
                                        torch.tensor(action_list),\
                                        torch.tensor(extrinsic_reward_list,dtype = torch.float),\
                                        torch.tensor(intrinsic_reward_list,dtype = torch.float),\
                                        torch.tensor(next_state_list,dtype=torch.float),\
                                        torch.tensor(extrinsic_done_list,dtype = torch.float),\
                                        torch.tensor(intrinsic_done_list,dtype = torch.float),\
                                        torch.tensor(prob_list,dtype = torch.float)
        if gpu:
            return s.cuda(),a.cuda(),er.cuda(),ir.cuda(),next_s.cuda(),extrinsic_done_list.cuda()\
            ,intrinsic_done_list.cuda(),prob.cuda() 
        else :
            return s,a,er,ir,next_s,extrinsic_done_list,intrinsic_done_list,prob  
    
    def train(self):
        state,action,extrinsic_reward,intrinsic_reward, next_state,extrinsic_done_list,\
        intrinsic_done_list,action_prob = self.make_batch()
        
        for i in range(K_epoch):
            state = state.squeeze()
            next_state = next_state.squeeze()
            predicted_action, predicted_extrinsic, predicted_intrinsic = self.ppo(state)
            predicted_next_action, predicted_next_extrinsic, predicted_next_intrinsic = self.ppo(next_state)

            if gpu:
                intrinsic_next_state_mean = torch.mean(torch.cat(list(self.intrinsic_input_queue.queue)),dim = 0).cuda()
                intrinsic_next_state_std = torch.std(torch.cat(list(model.intrinsic_input_queue.queue)),dim = 0).cuda()
            
                preprocessed_next_state = torch.clamp(((next_state - intrinsic_next_state_mean) / \
                                                   intrinsic_next_state_std + torch.tensor(1e-8).cuda()),-5,5)
            else:
                intrinsic_next_state_mean = torch.mean(torch.cat(list(self.intrinsic_input_queue.queue)),dim = 0)
                intrinsic_next_state_std = torch.std(torch.cat(list(model.intrinsic_input_queue.queue)),dim = 0)
            
                preprocessed_next_state = torch.clamp(((next_state - intrinsic_next_state_mean) / \
                                                   intrinsic_next_state_std + torch.tensor(1e-8)),-5,5)
            
            predict_feature, target_feature = self.rnd(next_state)
            td_error = extrinsic_reward + extrinsic_gamma * predicted_next_extrinsic * extrinsic_done_list
            delta = td_error - predicted_next_extrinsic
            if gpu:
                delta = delta.detach().cpu().numpy()
            else:
                delta = delta.detach().numpy()
            advantage_list = []
            
            advantage = 0.0
            for delta_t in delta[::-1]:
                advantage = gamma * lmbda * advantage + delta_t[0]
                advantage_list.append([advantage])
            advantage_list.reverse()
            if gpu:
                advantage = torch.tensor(advantage_list,dtype = torch.float).cuda()
            else:
                advantage = torch.tensor(advantage_list,dtype = torch.float)
            ##intrinsic_advantage
            intrinsic_td_error = intrinsic_reward + intrinsic_gamma * predicted_next_intrinsic * intrinsic_done_list
            intrinsic_delta = intrinsic_td_error - predicted_next_intrinsic
            if gpu:
                intrinsic_delta = intrinsic_delta.detach().cpu().numpy()
            else:
                intrinsic_delta = intrinsic_delta.detach().numpy()
                
            intrinsic_advantage_list = []
            intrinsic_advantage = 0.0
            
            for intrinsic_delta_t in intrinsic_delta[::-1]:
                intrinsic_advantage = gamma * lmbda * intrinsic_advantage + intrinsic_delta_t[0]
                intrinsic_advantage_list.append([intrinsic_advantage])
            intrinsic_advantage_list.reverse()
            if gpu:
                intrinsic_advantage = torch.tensor(intrinsic_advantage_list,dtype = torch.float).cuda()
            else:
                intrinsic_advantage = torch.tensor(intrinsic_advantage_list,dtype = torch.float)
            #### intrinsic_error
            intrinsic_error = (intrinsic_td_error - predicted_intrinsic.detach()).pow(2)
            if gpu:
                masking = torch.rand(len(intrinsic_error)).cuda()
                masking = (masking < update_proportion).type(torch.FloatTensor).cuda()
            else:
                masking = torch.rand(len(intrinsic_error))
                masking = (masking < update_proportion).type(torch.FloatTensor)
            if gpu:
                intrinsic_error = (intrinsic_error * masking).sum() / torch.max(intrinsic_error.sum(), torch.Tensor([1]).cuda())
            else:
                intrinsic_error = (intrinsic_error * masking).sum() / torch.max(masking.sum(), torch.Tensor([1]))

            
            now_action = predicted_action
            m = Categorical(now_action)
            entropy = m.entropy().mean()
            
            
            now_action = now_action.gather(1,action)
            
            
            ratio = torch.exp(torch.log(now_action) - torch.log(action_prob))
            advantage = extrinsic_advantage_coef * advantage +  intrinsic_advantage_coef * intrinsic_advantage
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio , 1-eps_clip, 1 + eps_clip) * advantage
            loss = - torch.min(surr1,surr2) + critic_coef * (F.smooth_l1_loss(predicted_extrinsic,td_error.detach()) +\
                    intrinsic_error) - ent_coef * entropy + F.mse_loss(predict_feature, target_feature)
            
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

In [6]:
epochs = 1000
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)


In [39]:
if gpu:
    model = Agent().cuda()
else:
    model = Agent()


In [8]:
'''
try:
    if gpu:
        model.load_state_dict(torch.load("weights/2_episode_model"))
    else:
        model.load_state_dict(torch.load("weights/2_episode_model", map_location={'cuda:0': 'cpu'}))
except :
    pass
'''

In [40]:
T_horizon = 20

for epoch in range(epochs):
    global_step = 0
    model.intrinsic_queue = queue.Queue()
    model.intrinsic_input_queue = queue.Queue()
    state = env.reset()
    state = np.array(state)/255
            #state = np.transpose(state,(2,0,1))
    state = np.moveaxis(state, -1, 0)
    state = torch.tensor(state).float()
    state = state.unsqueeze(0)
    done = False
    while not done :
        for t in range(T_horizon):
            #env.render()
            global_step +=1

            if gpu:
                action_prob, _ , _ = model.ppo.forward(state.cuda())
            else:
                action_prob, _ , _ = model.ppo.forward(state)
            m = Categorical(action_prob)
            action = m.sample().item()
            
            
            next_state, extrinsic_reward, done, info = env.step(action)
            next_state = np.array(next_state)/255
            next_state = np.moveaxis(next_state,-1,0)
            next_state = torch.tensor(next_state).float()
            next_state = next_state.unsqueeze(0)
            
            
            model.intrinsic_input_queue.put(next_state)
            if len(model.intrinsic_input_queue.queue) > 128:
                model.intrinsic_input_queue.get()
            intrinsic_next_state_mean = \
                    torch.mean(torch.cat(list(model.intrinsic_input_queue.queue)),dim = 0)
            if len(model.intrinsic_input_queue.queue) == 1:
                intrinsic_next_state_std = torch.zeros(1)
            else:
                intrinsic_next_state_std = \
                        torch.std(torch.cat(list(model.intrinsic_input_queue.queue)),dim = 0)
    
            preprocessed_next_state = \
                    torch.clamp(((next_state - intrinsic_next_state_mean) / (intrinsic_next_state_std + 1e-8)), -5,5)
            
            
            #(model.intrinsic_input_queue)
            if gpu:
                predictor,target = model.rnd.forward(preprocessed_next_state.cuda())
            else:
                predictor,target = model.rnd.forward(preprocessed_next_state)
            intrinsic_reward = (predictor - target).pow(2).sum(1) / 2
            if len(model.intrinsic_queue.queue) > 128:
                model.intrinsic_queue.get()
            print('intrinsic_reward.item() : ', intrinsic_reward.item())
            model.intrinsic_queue.put(intrinsic_reward.item())
            intrinsic_mean = np.mean(model.intrinsic_queue.queue)
            intrinsic_std = np.std(model.intrinsic_queue.queue)
            intrinsic_reward = (intrinsic_reward - intrinsic_mean) / (intrinsic_std+ 1e-8)
            
            if info['time'] == 0 or info['life'] == 1 or info['time'] == 300:
                done = True
                reward = -10.

            model.put_data((state.tolist(), action, extrinsic_reward/100, (intrinsic_reward.item()), next_state.tolist(), action_prob[0][action].item(), done))
            print('global_step : ',global_step,', action : ', action,'action_prob : ',action_prob.tolist()[0])
            print('extrinsic_reward : ',extrinsic_reward,'intrinsic_reward : ',intrinsic_reward.item())
            print('place',info['x_pos'])
            print('info',info)
            if done :
                print('epoch : ',epoch, ', global_step : ',global_step)
                break
            state = next_state
        model.train()
    #env.render()

#env.close()

intrinsic_reward.item() :  0.0
global_step :  1 , action :  3 action_prob :  [0.14285677671432495, 0.14285710453987122, 0.14285744726657867, 0.14285707473754883, 0.14285728335380554, 0.14285729825496674, 0.14285695552825928]
extrinsic_reward :  0 intrinsic_reward :  0.0
place 40
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}
intrinsic_reward.item() :  0.0
global_step :  2 , action :  2 action_prob :  [0.14285677671432495, 0.14285710453987122, 0.14285744726657867, 0.14285707473754883, 0.14285728335380554, 0.14285729825496674, 0.14285695552825928]
extrinsic_reward :  0 intrinsic_reward :  0.0
place 40
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}
intrinsic_reward.item() :  1.9368853569030762
global_step :  3 , action :  1 action_prob :  [0.14285677671432495, 0.

intrinsic_reward.item() :  22939.35546875
global_step :  19 , action :  0 action_prob :  [0.14285677671432495, 0.14285710453987122, 0.14285744726657867, 0.14285707473754883, 0.14285728335380554, 0.14285729825496674, 0.14285694062709808]
extrinsic_reward :  -1 intrinsic_reward :  2.2297821044921875
place 48
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 48, 'x_pos_screen': 48, 'y_pos': 79}
intrinsic_reward.item() :  31892.9140625
global_step :  20 , action :  3 action_prob :  [0.14285677671432495, 0.14285710453987122, 0.14285744726657867, 0.14285707473754883, 0.14285728335380554, 0.14285729825496674, 0.14285695552825928]
extrinsic_reward :  1 intrinsic_reward :  2.7246546745300293
place 49
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 49, 'x_pos_screen': 49, 'y_pos': 79}
intrinsic_reward.item() :  37568.2734375
global_step :  21 , a

intrinsic_reward.item() :  42682.17578125
global_step :  37 , action :  0 action_prob :  [0.12560036778450012, 0.12549930810928345, 0.1909065693616867, 0.12511563301086426, 0.12487104535102844, 0.184056356549263, 0.1239507645368576]
extrinsic_reward :  1 intrinsic_reward :  1.0110849142074585
place 65
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 65, 'x_pos_screen': 65, 'y_pos': 91}
intrinsic_reward.item() :  15310.85546875
global_step :  38 , action :  6 action_prob :  [0.12560328841209412, 0.12550222873687744, 0.19089840352535248, 0.12511862814426422, 0.1248740553855896, 0.1840495616197586, 0.12395390123128891]
extrinsic_reward :  1 intrinsic_reward :  -0.5207064747810364
place 66
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 66, 'x_pos_screen': 66, 'y_pos': 86}
intrinsic_reward.item() :  15174.3798828125
global_step :  39 , act

KeyboardInterrupt: 

In [22]:
model.intrinsic_queue.queue

deque([nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan,
       nan])

In [20]:
preprocessed_next_state

tensor([[[[-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          ...,
          [-5.,  5.,  5.,  ...,  5.,  5., nan],
          [-5.,  5.,  5.,  ...,  5., nan, nan],
          [ 5., nan, nan,  ..., nan, nan,  5.]],

         [[-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          ...,
          [-5., -5., -5.,  ..., -5., -5., nan],
          [-5., -5., -5.,  ..., -5., nan, nan],
          [-5., nan, nan,  ..., nan, nan, -5.]],

         [[-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          [-5., -5., -5.,  ..., -5., -5., -5.],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]]])

319

In [44]:
env.close()

ValueError: env has already been closed.

In [43]:
if gpu:
    model.load_state_dict(torch.load("weights/1_episode_model"))
else:
    model.load_state_dict(torch.load("weights/1_episode_model", map_location={'cuda:0': 'cpu'}))

In [55]:
epochs = 1000
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)


In [56]:
T_horizon = 20

for epoch in range(epochs):
    global_step = 0
    state = env.reset()
    state = np.array(state)/255
            #state = np.transpose(state,(2,0,1))
    state = np.moveaxis(state, -1, 0)
    state = torch.tensor(state).float()
    state = state.unsqueeze(0)
    done = False
    while not done :
        for t in range(T_horizon):
            env.render()
            global_step +=1

            if gpu:
                action_prob, _ , _ = model.ppo.forward(state.cuda())
            else:
                action_prob, _ , _ = model.ppo.forward(state)
            m = Categorical(action_prob)
            action = m.sample().item()
            next_state, extrinsic_reward, done, info = env.step(action)
            next_state = np.array(next_state)/255
            next_state = np.moveaxis(next_state,-1,0)
            next_state = torch.tensor(next_state).float()
            next_state = next_state.unsqueeze(0)
            if gpu:
                predictor,target = model.rnd.forward(next_state.cuda())
            else:
                predictor,target = model.rnd.forward(next_state)
            intrinsic_reward = (predictor - target).pow(2).sum(1) / 2
            if info['time'] == 0 :
                done = True
                reward = -10.
            if done :
                print('epoch : ',epoch, ', global_step : ',global_step)
                break
            print('global_step : ',global_step,', action : ', action,'action_prob : ',action_prob.tolist()[0])
            print('extrinsic_reward : ',extrinsic_reward,'intrinsic_reward : ',intrinsic_reward.item())
            print('place',info['x_pos'])
            print('info',info)
            state = next_state
    #env.render()

#env.close()

global_step :  1 , action :  2 action_prob :  [0.0008799101924523711, 0.017144275829195976, 0.9469815492630005, 0.00012634025188162923, 0.03220308944582939, 0.0026617946568876505, 3.0889298159308964e-06]
extrinsic_reward :  0 intrinsic_reward :  2.895847797393799
place 40
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 79}
global_step :  2 , action :  2 action_prob :  [0.0008799101924523711, 0.017144275829195976, 0.9469815492630005, 0.00012634025188162923, 0.03220308944582939, 0.0026617946568876505, 3.0889298159308964e-06]
extrinsic_reward :  0 intrinsic_reward :  2.895847797393799
place 40
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'x_pos_screen': 40, 'y_pos': 84}
global_step :  3 , action :  2 action_prob :  [0.0008799101924523711, 0.017144275829195976, 0.9469815492630005, 0.0001263402518816

global_step :  21 , action :  2 action_prob :  [0.000872174627147615, 0.01705988682806492, 0.9472266435623169, 0.0001249100751010701, 0.03207119181752205, 0.0026422275695949793, 3.0392150165425846e-06]
extrinsic_reward :  1 intrinsic_reward :  2.776517868041992
place 54
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 54, 'x_pos_screen': 54, 'y_pos': 140}
global_step :  22 , action :  2 action_prob :  [0.0008717708988115191, 0.017055492848157883, 0.9472392797470093, 0.00012483529280871153, 0.03206433728337288, 0.0026412021834403276, 3.0366250030056108e-06]
extrinsic_reward :  1 intrinsic_reward :  2.7469706535339355
place 55
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 55, 'x_pos_screen': 55, 'y_pos': 141}
global_step :  23 , action :  2 action_prob :  [0.0008705118671059608, 0.01704166643321514, 0.9472795128822327, 0.00012460377183

global_step :  40 , action :  2 action_prob :  [0.0008265043725259602, 0.016554201021790504, 0.9486960768699646, 0.00011654989793896675, 0.03127767890691757, 0.002526203403249383, 2.7533990305528278e-06]
extrinsic_reward :  2 intrinsic_reward :  4.443122863769531
place 86
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 398, 'world': 1, 'x_pos': 86, 'x_pos_screen': 83, 'y_pos': 99}
global_step :  41 , action :  2 action_prob :  [0.0008534396183677018, 0.01685388572514057, 0.9478251338005066, 0.00012146555673098192, 0.03174838051199913, 0.002594770397990942, 2.920414317486575e-06]
extrinsic_reward :  2 intrinsic_reward :  3.935343027114868
place 88
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 398, 'world': 1, 'x_pos': 88, 'x_pos_screen': 84, 'y_pos': 94}
global_step :  42 , action :  2 action_prob :  [0.0008298245957121253, 0.016591863706707954, 0.9485860466957092, 0.00011714545689756

global_step :  59 , action :  2 action_prob :  [0.0009168441756628454, 0.017543112859129906, 0.945823073387146, 0.00013320992002263665, 0.03282570466399193, 0.0027546973433345556, 3.3312371670035645e-06]
extrinsic_reward :  1 intrinsic_reward :  4.022909164428711
place 119
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 119, 'x_pos_screen': 97, 'y_pos': 79}
global_step :  60 , action :  2 action_prob :  [0.000850738724693656, 0.016824044287204742, 0.9479116797447205, 0.00012097017315682024, 0.03170161694288254, 0.002587905852124095, 2.9034567887720186e-06]
extrinsic_reward :  2 intrinsic_reward :  4.999667167663574
place 121
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 121, 'x_pos_screen': 98, 'y_pos': 79}
global_step :  61 , action :  2 action_prob :  [0.0009440652211196721, 0.017831934615969658, 0.9449849724769592, 0.000138333867

global_step :  80 , action :  2 action_prob :  [0.0008579960558563471, 0.0169041957706213, 0.9476789832115173, 0.00012230098946020007, 0.031827304512262344, 0.002606323454529047, 2.949099780380493e-06]
extrinsic_reward :  2 intrinsic_reward :  6.551303386688232
place 156
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 396, 'world': 1, 'x_pos': 156, 'x_pos_screen': 112, 'y_pos': 106}
global_step :  81 , action :  2 action_prob :  [0.0008757184259593487, 0.017098763957619667, 0.9471136331558228, 0.0001255619281437248, 0.03213223069906235, 0.0026511538308113813, 3.0618948585470207e-06]
extrinsic_reward :  2 intrinsic_reward :  5.705742835998535
place 158
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 396, 'world': 1, 'x_pos': 158, 'x_pos_screen': 112, 'y_pos': 109}
global_step :  82 , action :  2 action_prob :  [0.0009422510047443211, 0.017813043668866158, 0.9450395703315735, 0.000137986

...

global_step :  717 , action :  2 action_prob :  [0.0009302587131969631, 0.017685597762465477, 0.945409893989563, 0.00013573530304711312, 0.0330466590821743, 0.002788425888866186, 3.4215061077702558e-06]
extrinsic_reward :  1 intrinsic_reward :  5.078805923461914
place 1136
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 365, 'world': 1, 'x_pos': 1136, 'x_pos_screen': 127, 'y_pos': 4}
global_step :  718 , action :  2 action_prob :  [0.0009030529763549566, 0.017394836992025375, 0.9462538957595825, 0.0001306389458477497, 0.03259420022368431, 0.002720137359574437, 3.239842044422403e-06]
extrinsic_reward :  -15 intrinsic_reward :  3.056934118270874
place 1138
info {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 365, 'world': 1, 'x_pos': 1138, 'x_pos_screen': 127, 'y_pos': 255}
global_step :  719 , action :  2 action_prob :  [0.000879438768606633, 0.01713913306593895, 0.9469964504241943, 0.000126

global_step :  738 , action :  2 action_prob :  [0.0008698486490175128, 0.01703440025448799, 0.9473005533218384, 0.00012448159395717084, 0.03203125670552254, 0.0026363511569797993, 3.0243556921050185e-06]
extrinsic_reward :  1 intrinsic_reward :  2.8043856620788574
place 53
info {'coins': 0, 'flag_get': False, 'life': 1, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 53, 'x_pos_screen': 53, 'y_pos': 139}
global_step :  739 , action :  2 action_prob :  [0.0008717903983779252, 0.017055679112672806, 0.9472387433052063, 0.0001248393818968907, 0.032064586877822876, 0.0026412573643028736, 3.0367591534741223e-06]
extrinsic_reward :  1 intrinsic_reward :  2.776517868041992
place 54
info {'coins': 0, 'flag_get': False, 'life': 1, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 54, 'x_pos_screen': 54, 'y_pos': 140}
global_step :  740 , action :  2 action_prob :  [0.0008717708988115191, 0.017055492848157883, 0.9472392797470093, 0.00012483

global_step :  757 , action :  2 action_prob :  [0.0008735475712455809, 0.01707489788532257, 0.9471831321716309, 0.00012516356946434826, 0.032094668596982956, 0.0026457018684595823, 3.048015059903264e-06]
extrinsic_reward :  2 intrinsic_reward :  5.012538433074951
place 84
info {'coins': 0, 'flag_get': False, 'life': 1, 'score': 0, 'stage': 1, 'status': 'small', 'time': 399, 'world': 1, 'x_pos': 84, 'x_pos_screen': 82, 'y_pos': 104}
global_step :  758 , action :  2 action_prob :  [0.0008265043725259602, 0.016554201021790504, 0.9486960768699646, 0.00011654989793896675, 0.03127767890691757, 0.002526203403249383, 2.7533990305528278e-06]
extrinsic_reward :  1 intrinsic_reward :  4.477341651916504
place 86
info {'coins': 0, 'flag_get': False, 'life': 1, 'score': 0, 'stage': 1, 'status': 'small', 'time': 398, 'world': 1, 'x_pos': 86, 'x_pos_screen': 83, 'y_pos': 99}
global_step :  759 , action :  1 action_prob :  [0.0008535117376595736, 0.016854675486683846, 0.9478229284286499, 0.00012147859

global_step :  778 , action :  2 action_prob :  [0.000850738724693656, 0.016824044287204742, 0.9479116797447205, 0.00012097017315682024, 0.03170161694288254, 0.002587905852124095, 2.9034567887720186e-06]
extrinsic_reward :  1 intrinsic_reward :  5.015350818634033
place 121
info {'coins': 0, 'flag_get': False, 'life': 1, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 121, 'x_pos_screen': 98, 'y_pos': 79}
global_step :  779 , action :  2 action_prob :  [0.0009444362367503345, 0.017835846170783043, 0.9449737071990967, 0.00013840387691743672, 0.03328033536672592, 0.002823813585564494, 3.5178936741431244e-06]
extrinsic_reward :  3 intrinsic_reward :  4.874264717102051
place 124
info {'coins': 0, 'flag_get': False, 'life': 1, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 124, 'x_pos_screen': 99, 'y_pos': 79}
global_step :  780 , action :  5 action_prob :  [0.0008434404735453427, 0.016743067651987076, 0.9481474161148071, 0.000119636

KeyboardInterrupt: 

In [57]:
env.close()